In [1]:
using DataFrames, DataFramesMeta, LinearAlgebra, Distributions, Optim, Revise, Plots, CSV, Serialization

In [2]:
const datadir = "/export/storage_adgandhi/MiscLi/factract";

In [3]:
string(@__DIR__) in LOAD_PATH || push!(LOAD_PATH, @__DIR__)
using BLPmodule; const m = BLPmodule;

#### Data

In [4]:
test1state = true
if test1state 
    data = DataFrame(CSV.File("$datadir/analysis/factract_FL.csv"));
else
    data = DataFrame(CSV.File("$datadir/analysis/factract.csv"));
end;

In [5]:
# This is the big dataframe with every fac-tract combination within 10km
# println(names(data))
df = @select(data, 
    :t = :tractid, 
    :j = :facid, 
    :q = :restot,
    :M = :mktpop,
    :d = :dist ./ 100, 
    :d2 = (:dist ./ 100) .^ 2,
    :x1 = :dchrppd,
    :x2 = :rnhrppd
);
sort!(df, :j)
first(df, 3)

,t,j,q,M,d,d2,x1,x2
,Int64,String7,Int64,Float64,Float64,Float64,Float64,Float64
1,12069030105,105001,80,194.1,0.085943,0.00738619,4.269,0.285
2,12069030203,105001,80,583.0,0.0539458,0.00291015,4.269,0.285
3,12069030204,105001,80,178.4,0.0128454,0.000165004,4.269,0.285


In [19]:
serialize("jls/df.jls", df);

---
#### Mean utilities

In [7]:
df = deserialize("df.jls");
firm_IDs_long = df.j
tract_IDs_long = df.t;
X = Matrix{Float64}(df[!, [:x1, :x2]])
D = Matrix{Float64}(df[!, [:d, :d2]])
Q = df.q
M = df.M;

In [8]:
pars = m.set_Pars(K = 2, nI = 100, β = [-1., -2.]);

---

In [21]:
ec = m.make_Economy(
    firm_IDs_long,
    tract_IDs_long,
    X,
    D,
    Q,
    M,
    pars
    );


Economy with 394 firms and 3110 tracts.


In [22]:
serialize("jls/ec.jls", ec)

---

In [11]:
ec = deserialize("ec.jls");

In [12]:
using BenchmarkTools

In [13]:
ec = deserialize("ec.jls");
it = 1
@benchmark m.compute_deltas(ec,max_iter=it, verbose=false)

BenchmarkTools.Trial: 87 samples with 1 evaluation.
 Range (min … max):  55.278 ms … 62.949 ms  ┊ GC (min … max): 0.00% … 7.44%
 Time  (median):     56.498 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   57.587 ms ±  2.313 ms  ┊ GC (mean ± σ):  2.08% ± 3.37%

     ▄▆▆██ █▆  ▄                               ▄               
  ▄▄██████████▄█▄▄▄▁▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▄▆▁█▄█▄▆▄▁▁▁▄▁▁▁▁▄█ ▁
  55.3 ms         Histogram: frequency by time        62.8 ms <

 Memory estimate: 27.89 MiB, allocs estimate: 77826.

In [14]:
ec = deserialize("ec.jls");
it = 10
@benchmark m.compute_deltas(ec,max_iter=it, verbose=false)

BenchmarkTools.Trial: 18 samples with 1 evaluation.
 Range (min … max):  286.576 ms … 299.460 ms  ┊ GC (min … max): 0.00% … 2.55%
 Time  (median):     294.899 ms               ┊ GC (median):    2.10%
 Time  (mean ± σ):   294.097 ms ±   4.281 ms  ┊ GC (mean ± σ):  1.56% ± 1.14%

  ▁ ▁    ▁▁          ▁           █      ▁█  █      ▁     █▁  ▁▁  
  █▁█▁▁▁▁██▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁██▁▁█▁▁▁▁▁▁█▁▁▁▁▁██▁▁██ ▁
  287 ms           Histogram: frequency by time          299 ms <

 Memory estimate: 76.68 MiB, allocs estimate: 413958.

In [15]:
ec = deserialize("ec.jls");
it = 1000
@benchmark m.compute_deltas(ec,max_iter=it, verbose=false)

BenchmarkTools.Trial: 1 sample with 1 evaluation.
 Single result which took 7.187 s (1.48% GC) to evaluate,
 with a memory estimate of 1.48 GiB, over 10311178 allocations.

In [23]:
ec = deserialize("ec.jls");
deltas = m.compute_deltas(ec, max_iter=1000, verbose=true)
serialize("jls/deltas.jls", deltas);

dist = 9.94398110076844e-7, iterations = 276


In [17]:
# this was the line that the memory profiler said was creating the vast majority of allocations
# ...which doesn't make sense because it's not even in the inner loop
# ...and it's the inner loop that's driving the allocations and times (according to benchmarks)
@time for t in ec.tracts
    t.abδ .= t.D * pars.nlcoefs
end

  0.083885 seconds (31.54 k allocations: 18.647 MiB, 88.06% compilation time)


___
continued in `gmm.ipynb`